# TYTX - Advanced Features

This notebook covers:
- Custom types (Extension Types)
- Pydantic integration
- Locale formatting
- Compact arrays

In [ ]:
from decimal import Decimal
from datetime import date, datetime, timezone

from genro_tytx import registry, as_typed_text, from_text

## 1. Custom Types (Extension Types)

TYTX allows registering custom types with the `~` prefix.

In [ ]:
# Define a custom type: Invoice
class Invoice:
    def __init__(self, number: str, amount: Decimal):
        self.number = number
        self.amount = amount
    
    def as_typed_text(self) -> str:
        """Serialize to TYTX format"""
        return f"{self.number}|{self.amount}"
    
    @staticmethod
    def from_typed_text(s: str) -> "Invoice":
        """Deserialize from TYTX format"""
        number, amount = s.split("|")
        return Invoice(number, Decimal(amount))
    
    def __repr__(self):
        return f"Invoice({self.number}, {self.amount})"

# Register the custom type
registry.register_class("INV", Invoice)
print("Type INV registered!")

In [ ]:
# Use the custom type
invoice = Invoice("INV-001", Decimal("1500.00"))

# Serialize
typed = as_typed_text(invoice)
print(f"Serialized: {typed}")

# Deserialize
restored = from_text(typed)
print(f"Restored: {restored}")
print(f"Type: {type(restored).__name__}")

In [ ]:
# Another example: GPS Coordinates
class GeoPoint:
    def __init__(self, lat: float, lon: float):
        self.lat = lat
        self.lon = lon
    
    def as_typed_text(self) -> str:
        return f"{self.lat},{self.lon}"
    
    @staticmethod
    def from_typed_text(s: str) -> "GeoPoint":
        lat, lon = s.split(",")
        return GeoPoint(float(lat), float(lon))
    
    def __repr__(self):
        return f"GeoPoint({self.lat}, {self.lon})"

registry.register_class("GEO", GeoPoint)

# Test
point = GeoPoint(40.7128, -74.0060)  # New York
typed = as_typed_text(point)
print(f"New York: {typed}")

restored = from_text(typed)
print(f"Lat: {restored.lat}, Lon: {restored.lon}")

## 2. Pydantic Integration

TYTX can automatically generate structs from Pydantic models.

In [ ]:
try:
    from pydantic import BaseModel, Field
    PYDANTIC_AVAILABLE = True
except ImportError:
    PYDANTIC_AVAILABLE = False
    print("Pydantic not installed. Install with: pip install pydantic")

In [ ]:
if PYDANTIC_AVAILABLE:
    # Define Pydantic model
    class Customer(BaseModel):
        name: str = Field(min_length=1, max_length=100, title="Customer Name")
        email: str = Field(pattern=r'^[^@]+@[^@]+$')
        balance: Decimal = Field(ge=0, title="Balance")
        active: bool = True
    
    # Generate TYTX schema from model
    schema, metadata = registry.struct_from_model(Customer)
    
    print("Generated schema:")
    print(f"  {schema}")
    print("\nMetadata:")
    for field, meta in metadata.items():
        print(f"  {field}: {meta}")

In [ ]:
if PYDANTIC_AVAILABLE:
    # Register directly
    registry.register_struct_from_model("PYCUST", Customer)
    
    # Verify
    print(f"PYCUST schema: {registry.get_struct('PYCUST')}")
    print(f"'name' metadata: {registry.get_struct_metadata('PYCUST', 'name')}")

In [ ]:
if PYDANTIC_AVAILABLE:
    # Reverse: generate Pydantic model from TYTX struct
    registry.register_struct(
        "PRODUCT_V2",
        '{"code": "T", "name": "T", "price": "N"}',
        metadata={
            "code": {"validate": {"min": 3, "max": 10}},
            "name": {"ui": {"label": "Product Name"}},
            "price": {"validate": {"min": 0}}
        }
    )
    
    # Generate Pydantic model
    ProductModel = registry.model_from_struct("PRODUCT_V2")
    
    # Use the generated model
    product = ProductModel(code="ABC123", name="Widget", price=Decimal("29.99"))
    print(f"Product: {product}")
    print(f"Type: {type(product).__name__}")

## 3. Compact Arrays

For homogeneous arrays, TYTX supports a compact format.

In [ ]:
# Array of integers
numbers = [1, 2, 3, 4, 5]

# Normal format (each element typed)
normal = as_typed_text(numbers)
print(f"Normal: {normal}")

# Compact format (type only once)
compact = as_typed_text(numbers, compact_array=True)
print(f"Compact: {compact}")

In [ ]:
# Array of Decimals
prices = [Decimal("10.50"), Decimal("20.75"), Decimal("30.99")]

compact = as_typed_text(prices, compact_array=True)
print(f"Compact prices: {compact}")

# Parse
restored = from_text(compact)
print(f"Restored: {restored}")
print(f"Element type: {type(restored[0]).__name__}")

In [ ]:
# Nested arrays
matrix = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]

compact = as_typed_text(matrix, compact_array=True)
print(f"Matrix: {compact}")

restored = from_text(compact)
print(f"Restored: {restored}")

## 4. Locale Formatting

In [ ]:
from genro_tytx import as_text

# Date
today = date.today()

# ISO format (default)
print(f"ISO: {as_text(today)}")

# Localized format (requires babel)
try:
    print(f"US: {as_text(today, format=True, locale='en_US')}")
    print(f"DE: {as_text(today, format=True, locale='de_DE')}")
except Exception as e:
    print(f"Locale formatting not available: {e}")

In [ ]:
# Decimal
amount = Decimal("1234567.89")

print(f"ISO: {as_text(amount)}")

try:
    print(f"US: {as_text(amount, format=True, locale='en_US')}")
    print(f"DE: {as_text(amount, format=True, locale='de_DE')}")
except Exception as e:
    print(f"Locale formatting not available: {e}")

## 5. Unregister Custom Types

In [ ]:
# Unregister custom type
registry.unregister_class("GEO")
print("Type GEO unregistered")

# Verify
try:
    point = GeoPoint(0, 0)
    typed = as_typed_text(point)
    print(f"Typed: {typed}")  # Will no longer have ~GEO type
except:
    print("Type no longer recognized")

In [ ]:
# Unregister struct
registry.unregister_struct("PRODUCT_V2")
print("Struct PRODUCT_V2 unregistered")

## 6. API Summary

### Text API
| Function | Description |
|----------|-------------|
| `from_text(s)` | Parse typed string -> Python |
| `as_text(v)` | Python -> string (no type) |
| `as_typed_text(v)` | Python -> string with type |

### JSON API
| Function | Description |
|----------|-------------|
| `from_json(s)` | JSON -> dict with hydrated types |
| `as_json(v)` | Python -> standard JSON |
| `as_typed_json(v)` | Python -> JSON with TYTX types |

### XML API
| Function | Description |
|----------|-------------|
| `from_xml(s)` | XML -> dict with hydrated types |
| `as_xml(v)` | Python -> standard XML |
| `as_typed_xml(v)` | Python -> XML with TYTX types |

### Registry API
| Function | Description |
|----------|-------------|
| `registry.register_class(code, cls)` | Register custom type |
| `registry.register_struct(code, schema)` | Register struct |
| `registry.struct_from_model(cls)` | Pydantic -> schema |
| `registry.model_from_struct(code)` | Schema -> Pydantic |

## Done!

These notebooks cover the main features of TYTX.

For more information:
- [GitHub Repository](https://github.com/genropy/genro-tytx)
- [Documentation](https://genro-tytx.readthedocs.io)